In [1]:
from dotenv import load_dotenv 

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


# FewShotPromptTemplate

- one-shot, few-shot: 답변의 예시를 보여주는 것이다.
- zero-shot의 경우에는 질문자의 의도를 정확히 파악하기 어렵다.

In [3]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3:8b", temperature=0)

question = "대한민국의 수도는 어디인가요?"

print(llm.invoke(question).content)

😊

The capital of South Korea is Seoul (,).


In [4]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

In [5]:
# Model에게 논리적인 추론과정을 보여준다.
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

In [6]:
# one-shot
example_prompt = PromptTemplate.from_template(
    """
Question:
{question}

Answer:
{answer}

"""
)

In [7]:
# one-shot
# 0번째 index의 examples를 unpacking해서 가져온다.
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

# 0번째 index의 examples를 unpacking해서 가져온다.
print(example_prompt.format(**examples[0]))

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [8]:
print(example_prompt.format(**examples[1]))

Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일



In [9]:
prompt = FewShotPromptTemplate(
    examples=examples,  # 예시
    example_prompt=example_prompt,  # 에시들을 template구성을 해서 넣어준다.
    suffix="Question:\n{question}\nAnswer:",  # 사용자의 질문과 answer:까지만 넣어준다. 나머지 answer은 model이 채워준다.
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살 인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살 인가요?
Answer:


In [10]:
print(llm.invoke(final_prompt).content)

This question requires additional questions to arrive at the answer. Here's how I would respond:

Additional question: What year was Google founded?
Intermediate answer: Google was founded in 1998.
Additional question: What is Bill Gates' birthdate?
Intermediate answer: Bill Gates was born on October 28, 1955.
Additional question: How old was Bill Gates when Google was founded?
Intermediate answer: Bill Gates was 43 years old when Google was founded (1998 - 1955 = 43).
Final answer: 43


In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import stream_response

In [12]:
# 동적으로 입력받기.
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

In [13]:
chain = prompt | llm | StrOutputParser()

In [14]:
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인강?"}
)
stream_response(answer)

This question requires additional questions to get the correct answer. Here's how I would respond:

Additional question: What year was Google founded?
Intermediate answer: Google was founded in 1998.
Additional question: What is Bill Gates' birthdate?
Intermediate answer: Bill Gates was born on October 28, 1955.
Additional question: How old was Bill Gates when Google was founded?
Intermediate answer: Bill Gates was 43 years old when Google was founded (1998 - 1955 = 43).
Final answer: 43

# Example Selector 
질문을 하면 유사도 계산을 하여, 유사도가 높은 몇개를 이용하자.

- FewShotPromptTemplate의 단점
1. 모든 prompt를 입력으로 받는다. 이것은 돈이다.
2. local model을 사용하면 context가 작다. prompt가 길어질수록 답변이 짧아진다.

반드시 OpenAI Embedding을 사용해야하나???

In [15]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-mcYdBRZz-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-mcYdBRZz-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [18]:
from langchain_community.vectorstores import Chroma

chroma = Chroma("example_selector", embeddings)

In [19]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,  # 입력 예시들
    embeddings,  # 임베딩 클래스
    Chroma,  # 임베딩 저장 및 유사성 검색하는 Vector DB
    k=1,  # 생성개수
)

### SemanticSimilarityExampleSelector

In [20]:
# 입력과 가장 유사한 예시 확인
selected_examples = example_selector.select_examples({"question": question})
qustion = "Google이 창립된 연도에 Bill Gates의 나이는 몇살인가요?"
print(f"입력에 가장 유사한 예시: \n{question}\n")
for example in selected_examples:
    print(f'question: \n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

입력에 가장 유사한 예시: 
Google이 창립된 연도에 Bill Gates의 나이는 몇 살 인가요?

question: 
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [21]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="question:\n{question}\nAnswer:",
    input_variables=["question"],
)

In [22]:
chain = prompt | llm

In [23]:
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇살인가요?"
print(chain.invoke(question).content)

추가 질문이 필요한가요: 예.
추가 질문: Google이 창립된 연도는 1998년입니다. Bill Gates의 나이는?
중간 답변: Bill Gates는 1945년에 태어났습니다. 따라서, 1998년에는 53세였습니다.
최종 답변은: 53


In [24]:
answer = chain.stream(question)
stream_response(answer)

추가 질문이 필요한가요: 예.
추가 질문: Google이 창립된 연도는 1998년입니다. Bill Gates의 나이는?
중간 답변: Bill Gates는 1945년에 태어났습니다. 따라서, 1998년에는 53세였습니다.
최종 답변은: 53

### MaxMarginalRelevanceExampleSelector
- 답변할 때, 유사성도 있지만, 다양한 예제를 가져오고 싶을때 mmr을 사용한다.
- 중복성을 줄이면서, 관련성이 높은 정보를 제공하는것을 목표로 한다. 중요한것은 다양성이다.
- 즉, 관련성과 다양성을 중요시한다.
- 람다에 의해서 값을 조절할수 있으며, 클수록 관련성을, 작을수록 다양성을 중시한다.

In [25]:
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector

In [26]:
chroma = Chroma("example_selector", embeddings)

In [27]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=1,
)

In [28]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer",
    input_variables=["question"],
)
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

Number of requested results 20 is greater than number of elements in index 8, updating n_results = 8


Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer


In [29]:
chain = prompt | llm

In [30]:
# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

Number of requested results 20 is greater than number of elements in index 8, updating n_results = 8


Another question! 🤔

추가 질문: Google이 창립된 연도는 1998년입니다. Bill Gates의 나이는...?

중간 답변: Bill Gates는 1945년에 태어났습니다.

계산: 1998 - 1945 = 53 years old

최종 답변은: Bill Gates was 53 years old when Google was founded in 1998. 📊

# FewShotChatMessagePromptMessage


In [31]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

In [32]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma, FAISS

In [33]:
chroma = Chroma("fewshot_chat", embeddings)

In [34]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=1,
)

In [35]:
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{instruction}: \n {input}"), ("ai", "{answer}")]
)

In [36]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector, example_prompt=example_prompt
)

In [37]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

In [38]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [39]:
chain = final_prompt | llm

In [40]:
answer = chain.stream(question)
stream_response(answer)

회의록: ABC 기술 회사의 모바일 애플리케이션 프로젝트 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사의 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 회의는 최현수 프로젝트 매니저의 개회사로 시작됨.
   - 회의의 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것.

2. 프로젝트 진행 상황 (황지연)
   - 황지연은 프로젝트의 현재 진행 상황을 업데이트.
   - 개발 진행도와 주요 이슈에 대한 설명.

3. UI/UX 디자인 업데이트 (김태영)
   - 김태영은 UI/UX 디자인의 현재 진행 상황을 업데이트.
   - 새로운 기능과 인터페이스에 대한 예시를 공유.

4. 다음 주 목표 설정
   - 팀원들은 다음 주까지의 목표를 설정.
   - 프로젝트의 주요 이슈와 해결 방안에 대한 논의.

5. 마일스톤 계획 (최현수)
   - 최현수는 다가오는 마일스톤에 대한 계획을 제시.
   - 프로젝트의 중요한 DEADLINE과 이를 달성하기 위한 전략에 대한 설명.

6. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 프로젝트의 진행 상황과 다음 주 목표를 확인.

7. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 황지연 담당.

# CustomExampleSelector
- instruction과, 별도로 준 instriction을 준것을 유사도 검색을 하면 어떨까?

기존 example example_selector에는 유사도검색에 문제점이있다.
유사도 계산시에 instruction과 input을 사용한다. 하지만, instruction만 사용할 시 유사도 검색이 제대로 되지 않는다.

In [41]:
question = {"instruction": "회의록을 작성해 주세요"}
example_selector.select_examples(question)

[{'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n'}]

In [42]:
from langchain_teddynote.prompts import CustomExampleSelector

custom_selector = CustomExampleSelector(examples, embeddings)
custom_selector.select_examples({"instruction": "다음 문장을 회의록 작성해 주세요"})

[{'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n'}]

In [43]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}: \n{input}"),
        ("ai", "{answer}"),
    ]
)

custom_fewshow_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector, example_prompt=example_prompt
)

In [44]:
custom_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [45]:
chain = custom_prompt | llm

In [46]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))

회의록: ABC 기술 회사의 제품 개발 팀 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사의 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 회의는 최현수 프로젝트 매니저의 개회사로 시작됨.
   - 회의의 목적은 새로운 모바일 애플리케이션 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것.

2. 프로젝트 진행 상황 (황지연)
   - 황지연은 프로젝트의 현재 진행 상황을 업데이트.
   - 개발 진행도와 주요 이슈에 대한 설명.

3. UI/UX 디자인 업데이트 (김태영)
   - 김태영은 UI/UX 디자인 작업의 진행 상황을 공유.
   - 새로운 기능과 인터페이스에 대한 예시를 제시.

4. 다음 주 목표 설정
   - 팀원들은 다음 주까지의 목표를 설정.
   - 프로젝트의 주요 이슈와 해결 방안에 대한 논의.

5. 마무리
   - 회의는 최현수 프로젝트 매니저의 마무리로 끝남.
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 황지연 담당.

회의록 주요 내용:

* 프로젝트 진행 상황: 80% 완료
* 다음 주 목표:
	+ 개발: 기능 A, B 완성
	+ UI/UX 디자인: 인터페이스 C, D 완성
* 다음 회의 날짜: 2023년 12월 30일

참고: 회의록은 프로젝트 진행 상황을 확인하고, 팀원들이 공유하는 정보를 정리한 문서입니다.

In [47]:
question = {
    "instruction": "문서를 요약해 주세요",
    "input": "이 문서는 '2023년 글로벌 경제 전망'에 관한 30페이지에 달하는 상세한 보고서입니다. 보고서는 세계 경제의 현재 상태, 주요 국가들의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해에 대한 경제 예측을 다룹니다. 이 보고서는 또한 다양한 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 분석하고 있습니다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))

Here is a summary of the document:

**Global Economic Outlook 2023**

* **Current State**: The report provides an overview of the current state of the global economy, including key indicators such as GDP growth rates and inflation rates.
* **Country-by-Country Analysis**: The report analyzes the economic performance of major countries, including the United States, China, Japan, and the European Union, highlighting their strengths and weaknesses.
* **Global Trade Trends**: The report examines the current state of global trade, including trends in international trade, tariffs, and supply chain disruptions.
* **2023 Economic Predictions**: The report provides predictions for the global economy in 2023, including forecasts for GDP growth rates, inflation rates, and interest rates.
* **Macroeconomic Factors**: The report analyzes various macroeconomic factors that will impact the global economy in 2023, such as monetary policy, fiscal policy, and demographic changes.
* **Geopolitical Risks*

In [48]:
question = {
    "instruction": "문장을 교정해 주세요",
    "input": "회사는 올해 매출이 증가할 것으로 예상한다. 새로운 전략이 잘 작동하고 있다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))

회사는 올해 매출이 증가할 것으로 예측하며, 새로운 전략의 효과적인 수행으로 인해 이를 달성할 수 있을 것이다.

# Langchain Hub

- hun을 통해 prompt를 받아올 수 있다.

In [49]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [50]:
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [51]:
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
    "주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:"
)
prompt

ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:'))])

In [ ]:
from langchain import hub

# 프롬프트를 허브에 업로드합니다.
hub.push("teddynote/simple-summary-korean", prompt)

In [53]:
from langchain import hub

# 프롬프트를 허브로부터 가져옵니다.
pulled_prompt = hub.pull("teddynote/simple-summary-korean")

In [54]:
# 프롬프트 내용 출력
print(pulled_prompt)

input_variables=['context'] metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'simple-summary-korean', 'lc_hub_commit_hash': 'b7e31df5666de7758d72fd038875973520d141548280185ee5b5ba846f015308'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:'))]


: 